# Recommending a game to a user

The steps below will take you through how to build your recommendation system. Before attempting this, please make sure you have followed the 03_01_MySQL_Setup.ipynb steps. 

In [ ]:
# Uncomment to install packages
# !pip install -r requirements.txt

## Querying MySQL in Python

In [ ]:
PASSWORD = '<your_MYSQL_instance_password>'
import mysql.connector

connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=PASSWORD,
  database="steam_data")

In [ ]:
sql = 'SELECT count(*) FROM steam_purchase;'
print(sql)

In [ ]:
cursor = connection.cursor()
cursor.execute(sql)
result = cursor.fetchall()
print(result)
connection.close()

## Method for querying

In [ ]:
%%writefile graphtastic/database/mysql.py
import mysql.connector
def query_mysql(query, password, host='localhost', user='root',database='steam_data'):
    ''''
    Query a MySQL database instance
 
    :param query: a SQL query to run against the MySQL database
    :param password: the MySQL instance password when setting up the solution
    :param host: the host (default=localhost)
    :param user: the user (default=root) for all the MySQL priviledges
    :param database: defaults to `steam_data` but this could be any database of choosing
    :return: SQL query results
    '''
    connection = mysql.connector.connect(
    host=host,
    user=user,
    passwd=password,
    database=database
    )
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [ ]:
from graphtastic.database.mysql import query_mysql

In [ ]:
result = query_mysql('SELECT count(*) FROM steam_purchase;',
                    password=PASSWORD)
print(result)

## Examing the data

In [ ]:
result = query_mysql('SELECT * FROM steam_play LIMIT 4;', password=PASSWORD)
for row in result:
	print(row)

In [ ]:
purchase_result = query_mysql('SELECT * FROM steam_purchase LIMIT 4;', password=PASSWORD)
for row in purchase_result:
	print(row)

## Path based analytics in tabular data

In [ ]:

result = query_mysql('SELECT game_name from steam_purchase WHERE id = "151603712";', password=PASSWORD)
print(result[:10])
print(len(result))


In [ ]:
games = ['"' + game[0] + '"' for game in result]
games_string = ','.join(games)
print(games_string)

In [ ]:
query_2 = f'SELECT id from steam_purchase WHERE game_name ' \
      	f'IN ({games_string}) AND id != "151603712";'
result_2 = query_mysql(query_2, password=PASSWORD)
users = [user[0] for user in result_2]
print(users[:10])
print(len(users))


In [ ]:
users = list(set(users))
print(users[:10])
print(len(users))


In [ ]:
users = ['"' + user + '"' for user in users]
users_string = ','.join(users)
query_3 = f'SELECT game_name from steam_purchase WHERE id IN ({users_string}) ' \
      	f'AND game_name NOT IN ({games_string});'
 
result_3 = query_mysql(query_3, password=PASSWORD)
recommended_games = [game[0] for game in result_3]
print(recommended_games[:10])


In [ ]:
from collections import Counter
game_frequency = Counter(recommended_games)
print(game_frequency)